In [ ]:
import pandas as pd 
from function import *
import glob
path = download_dataset('birds')[0]
df = pd.read_csv(path)
train_data_path = df[df['data set'] == 'train'].reset_index(drop=True)
val_data_path = df[df['data set'] == 'valid'].reset_index(drop=True)

test_path = path.replace('birds.csv', 'test')
test_data_path = glob.glob(f"{test_path}/*.jpg")
test_data_path = pd.DataFrame(test_data_path, columns=["filepaths"])

In [ ]:
# Divide dataset for comfortable classes
from sklearn.model_selection import train_test_split
train_data_path, _ = train_test_split(
    train_data_path, 
    test_size=0.9, 
    stratify=train_data_path['class id'],
    random_state=42
)

In [ ]:
from autogluon.multimodal.utils.misc import path_expander

PATH_COL = 'filepaths'
base_path = path.replace('birds.csv', '')
train_data_path[PATH_COL] = train_data_path[PATH_COL].apply(lambda ele: path_expander(ele, base_folder='./birds_525_species/'))
val_data_path[PATH_COL] = val_data_path[PATH_COL].apply(lambda ele: path_expander(ele, base_folder='./birds_525_species/'))
test_data_path[PATH_COL] = test_data_path[PATH_COL].apply(lambda ele: path_expander(ele, base_folder='./birds_525_species/'))
print(val_data_path.head())

In [ ]:
from autogluon.multimodal import MultiModalPredictor
model_path = generate_path('./AutogluonModels/', 'Birds')
predictor = MultiModalPredictor(label='class id', problem_type='multiclass', path = model_path, presets='medium_quality')
predictor.fit(train_data=train_data_path,
              tuning_data=val_data_path,
              hyperparameters={"env.batch_size": 128},
             )

In [ ]:
scores = predictor.evaluate(val_data_path, metrics=["accuracy"])
print('Top-1 test acc: %.3f' % scores["accuracy"])